# Import Libraries

In [1]:
!pip install keras-cv-attention-models

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 613.4/613.4 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 37.6 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 21.12.2 requires cupy-cuda115, which is not installed.
tfx-bsl 1.12.0 requires google-api-python-client<2,>=1.7.11, but you have google-api-python-client 2.79.0 which is incompatible.
tfx-bsl 1.12.0 requires pyarrow<7,>=6, but you have pyarrow 5.0.0 which is incompatible.
tensorflow-transform 1.12.0 requires pyarrow<7,>=6, but you have pyarrow 5.0.0 which is incompatible.
onnx 1.13.1 requires protobuf<4,>=3.20.2, but you have protobuf 3.19.6 which is incompatible.
apache-beam 2.44.0 requires 

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_addons as tfa
import matplotlib.pyplot as plt
import matplotlib as mpl

from tqdm.notebook import tqdm
from multiprocessing import cpu_count
from kaggle_datasets import KaggleDatasets
from sklearn.model_selection import train_test_split
from keras_cv_attention_models import convnext


import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds

import os
import time
import pickle
import math
import random
import sys
import cv2
import gc
import datetime

print(f'Tensorflow Version: {tf.__version__}')
print(f'Python Version: {sys.version}')

Tensorflow Version: 2.11.0
Python Version: 3.7.12 | packaged by conda-forge | (default, Oct 26 2021, 06:08:53) 
[GCC 9.4.0]


In [3]:
now = datetime.datetime.now().strftime("%d-%b-%Y %H-%M-%S")
np.save(now, np.array([now]))

## MP policy

In [4]:
# # float32 or mixed_float16 (mixed precision: compute float16, variable float32)
# # TPU is fast enough and has enough memory to use float32
# policy = tf.keras.mixed_precision.Policy('float32')
# tf.keras.mixed_precision.set_global_policy(policy)

# print(f'Compute dtype: {tf.keras.mixed_precision.global_policy().compute_dtype}')
# print(f'Variable dtype: {tf.keras.mixed_precision.global_policy().variable_dtype}')

## Matplotlib Config¶


In [5]:
# MatplotLib Global Settings
mpl.rcParams.update(mpl.rcParamsDefault)
mpl.rcParams['xtick.labelsize'] = 16
mpl.rcParams['ytick.labelsize'] = 16
mpl.rcParams['axes.labelsize'] = 18
mpl.rcParams['axes.titlesize'] = 24

## Config

In [6]:
# gpu_devices = tf.config.experimental.list_physical_devices('GPU')
# for device in gpu_devices:
#     tf.config.experimental.set_memory_growth(device, True)

## Load Data

In [7]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  '/kaggle/input/fake-images-detecttion-dataset/Keras_Data-20230318T161559Z-001/Keras_Data/Train_Data',
  seed=0,
  image_size=(160, 160),
  batch_size=16)

Found 47995 files belonging to 2 classes.


In [8]:
valid_ds = tf.keras.utils.image_dataset_from_directory(
  '/kaggle/input/fake-images-detecttion-dataset/Keras_Data-20230318T161559Z-001/Keras_Data/Validation_Data',
  seed=0,
  image_size=(160, 160),
  batch_size=8)

Found 6000 files belonging to 2 classes.


In [9]:
test_ds = tf.keras.utils.image_dataset_from_directory(
  '/kaggle/input/fake-images-detecttion-dataset/Keras_Data-20230318T161559Z-001/Keras_Data/Test_Data',
  seed=0,
  image_size=(160, 160),
  batch_size=8)

Found 6001 files belonging to 2 classes.


In [10]:
# normalization_layer = tf.keras.layers.Rescaling(1./255)
# normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))


In [11]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = valid_ds.prefetch(buffer_size=AUTOTUNE)

# Model

In [12]:
# Seed all random number generators
def seed_everything(seed=0):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

seed_everything()


In [13]:
def normalize(image):
#     # Repeat channels to create 3 channel images required by pretrained ConvNextV2 models
#     image = tf.repeat(image, repeats=3, axis=3)
    # Cast to float 32
    image = tf.cast(image, tf.float32)
    # Normalize with respect to ImageNet mean/std
    image = tf.keras.applications.imagenet_utils.preprocess_input(image, mode='torch')

    return image

In [14]:
STRATEGY = tf.distribute.MirroredStrategy() # default distribution strategy in Tensorflow. Works on CPU and single GPU.

N_REPLICAS = STRATEGY.num_replicas_in_sync

In [15]:
N_REPLICAS

2

In [16]:
def get_model():
#     # Verify Mixed Policy Settings
#     print(f'Compute dtype: {tf.keras.mixed_precision.global_policy().compute_dtype}')
#     print(f'Variable dtype: {tf.keras.mixed_precision.global_policy().variable_dtype}')
    INPUT_SHAPE = (160, 160, 3)
    
    with STRATEGY.scope():
        # Set seed for deterministic weights initialization
        seed_everything()
        
        # Inputs, note the names are equal to the dictionary keys in the dataset
        image = tf.keras.layers.Input(INPUT_SHAPE, name='image', dtype=tf.uint8)
        
       
        
        # Normalize Input
        image_norm = normalize(image)

        # CNN Prediction in range [0,1]
        x = convnext.ConvNeXtV2Tiny(
            input_shape=(160, 160, 3),
            pretrained='imagenet21k-ft1k',
            num_classes=0,
        )(image_norm)
        
        # Average Pooling BxHxWxC -> BxC
        x = tf.keras.layers.GlobalAveragePooling2D()(x)
        # Dropout to prevent Overfitting
        x = tf.keras.layers.Dropout(0.50)(x)
        # Output value between [0, 1] using Sigmoid function
        outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

        # We will use the famous AdamW optimizer for fast learning with weight decay
        optimizer = tfa.optimizers.AdamW(learning_rate=LR_MAX, weight_decay=LR_MAX*WD_RATIO, epsilon=1e-6)

        # Loss
        loss = tf.keras.losses.BinaryCrossentropy(from_logits=False)
        
        # Metrics
        metrics = [
            tfa.metrics.F1Score(num_classes=1),
            tf.keras.metrics.Precision(),
            tf.keras.metrics.Recall(),
            tf.keras.metrics.AUC(),
            tf.keras.metrics.BinaryAccuracy(),
        ]

        model = tf.keras.models.Model(inputs=image, outputs=outputs)
        
        model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

        return model

## Lr scheduler

In [17]:
N_EPOCHS=3
VERBOSE=1 
N_REPLICAS =2
LR_MAX = 5e-6 * N_REPLICAS
WD_RATIO = 1e-5

N_WARMUP_EPOCHS = 0

In [18]:
# Learning rate scheduler with logaritmic warmup and cosine decay
def lrfn(current_step, num_warmup_steps, lr_max, num_cycles=0.50, num_training_steps=N_EPOCHS):
    
    if current_step < num_warmup_steps:
        return lr_max * 0.10 ** (num_warmup_steps - current_step)
    else:
        progress = float(current_step - num_warmup_steps) / float(max(1, num_training_steps - num_warmup_steps))

        return max(0.0, 0.5 * (1.0 + math.cos(math.pi * float(num_cycles) * 2.0 * progress))) * lr_max

## weight decay

In [19]:
# Tensorflow Learning Rate Scheduler does not update weight decay, need to do it manually in a custom callback
class WeightDecayCallback(tf.keras.callbacks.Callback):
    def __init__(self, wd_ratio=WD_RATIO):
        self.step_counter = 0
        self.wd_ratio = wd_ratio
    
    def on_epoch_begin(self, epoch, logs=None):
        model.optimizer.weight_decay = model.optimizer.learning_rate * self.wd_ratio
        print(f'learning rate: {model.optimizer.learning_rate.numpy():.2e}, weight decay: {model.optimizer.weight_decay.numpy():.2e}')

# Training

In [20]:

# Learning Rate 
# Learning rate for encoder
LR_SCHEDULE = [lrfn(step, num_warmup_steps=N_WARMUP_EPOCHS, lr_max=LR_MAX, num_cycles=0.50) for step in range(N_EPOCHS)]
lr_callback = tf.keras.callbacks.LearningRateScheduler(lambda step: LR_SCHEDULE[step], verbose=0)
model = get_model()

    

115145456/115145456 [==============================] - 2s 0us/step
>>>> Load pretrained from: /root/.keras/models/convnext_v2_tiny_224_imagenet21k-ft1k.h5


In [21]:
history = model.fit(
            train_ds,            
            validation_data = valid_ds,
            epochs = N_EPOCHS,
            verbose = VERBOSE,
            callbacks = [
                lr_callback,
                WeightDecayCallback(),
            ],
           
        )

learning rate: 1.00e-05, weight decay: 1.00e-10
Epoch 1/3


2023-03-19 21:51:22.226629: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel/convnext_v2_tiny/stack1_block2_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


3000/3000 [==============================] - 713s 222ms/step - loss: 0.4227 - f1_score: 0.6667 - precision: 0.8110 - recall: 0.8082 - auc: 0.8937 - binary_accuracy: 0.8099 - val_loss: 0.1699 - val_f1_score: 0.6667 - val_precision: 0.9232 - val_recall: 0.9500 - val_auc: 0.9823 - val_binary_accuracy: 0.9355 - lr: 1.0000e-05
learning rate: 7.50e-06, weight decay: 7.50e-11
Epoch 2/3
3000/3000 [==============================] - 611s 204ms/step - loss: 0.1523 - f1_score: 0.6667 - precision: 0.9397 - recall: 0.9441 - auc: 0.9853 - binary_accuracy: 0.9418 - val_loss: 0.1060 - val_f1_score: 0.6667 - val_precision: 0.9693 - val_recall: 0.9590 - val_auc: 0.9922 - val_binary_accuracy: 0.9643 - lr: 7.5000e-06
learning rate: 2.50e-06, weight decay: 2.50e-11
Epoch 3/3
3000/3000 [==============================] - 600s 200ms/step - loss: 0.0842 - f1_score: 0.6667 - precision: 0.9670 - recall: 0.9693 - auc: 0.9952 - binary_accuracy: 0.9681 - val_loss: 0.0906 - val_f1_score: 0.6667 - val_precision: 0.972

In [22]:
model.save_weights('model.h5')

# Test

In [23]:
test_ds = test_ds.prefetch(buffer_size=AUTOTUNE)

In [24]:
model.evaluate(test_ds)

751/751 [==============================] - 49s 64ms/step - loss: 0.0880 - f1_score: 0.6666 - precision: 0.9710 - recall: 0.9693 - auc: 0.9945 - binary_accuracy: 0.9702


[0.08800782263278961,
 array([0.6665926], dtype=float32),
 0.9709515571594238,
 0.9693333506584167,
 0.9944921731948853,
 0.9701716303825378]

In [25]:
y_pred = []  # store predicted labels
y_true = []  # store true labels

# iterate over the dataset
for image_batch, label_batch in test_ds:   # use dataset.unbatch() with repeat
   # append true labels
   y_true.append(label_batch)
   # compute predictions
   preds = model.predict(image_batch)
   # append predicted labels
   preds[preds <= 0.5] = 0.
   preds[preds > 0.5] = 1.
   y_pred.append(preds)

# convert the true and predicted labels into tensors
correct_labels = tf.concat([item for item in y_true], axis = 0)
predicted_labels = tf.concat([item for item in y_pred], axis = 0)

1/1 [==============================] - 0s 155ms/step


In [26]:
import matplotlib.pyplot as plt
import seaborn as sns
# from sklearn import confusion_matrix

# cm = confusion_matrix(correct_labels.numpy(), predicted_labels.numpy())

In [27]:
# cm

In [28]:
from sklearn.metrics import classification_report
classification_report(correct_labels.numpy(), predicted_labels.numpy())

'              precision    recall  f1-score   support\n\n           0       0.97      0.97      0.97      3001\n           1       0.97      0.97      0.97      3000\n\n    accuracy                           0.97      6001\n   macro avg       0.97      0.97      0.97      6001\nweighted avg       0.97      0.97      0.97      6001\n'